In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain, SequentialChain

# Configurar el modelo OpenAI
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Configurar el modelo Ollama
llm = ChatOllama(model="gemma:2b", temperature=0.7)

# Paso 1 - Prompt de generación con Chain of Thought
system_template = "Eres un escritor creativo que razona paso a paso."
human_template = (
    "Piensa en pasos lógicos y escribe un texto de 5 líneas "
    "sobre '{topic}' en estilo {style}."
)

system_msg_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_msg_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_msg_prompt, human_msg_prompt])

generate_chain = LLMChain(llm=llm, prompt=chat_prompt, output_key="generated_text")

# Paso 2 - Refinar el texto generado
refine_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Ahora eres un editor que pule textos."),
    HumanMessagePromptTemplate.from_template(
        "Pulir el siguiente texto para mejorar claridad y estilo:\n\n{generated_text}"
    )
])
refine_chain = LLMChain(llm=llm, prompt=refine_prompt, output_key="refined_text")

# Cadena secuencial
chain = SequentialChain(
    chains=[generate_chain, refine_chain],
    input_variables=["topic", "style"],
    output_variables=["generated_text", "refined_text"],
    verbose=True,
)

# Ejecutar el chain
outputs = chain({"topic": "la luna", "style": "poético y evocativo"})
print("Texto generado:\n", outputs["generated_text"])
print("\nTexto refinado:\n", outputs["refined_text"])


/tmp/ipykernel_20895/2212766284.py:24: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  generate_chain = LLMChain(llm=llm, prompt=chat_prompt, output_key="generated_text")
/tmp/ipykernel_20895/2212766284.py:44: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  outputs = chain({"topic": "la luna", "style": "poético y evocativo"})




> Entering new SequentialChain chain...

> Finished chain.
Texto generado:
 La luna, un satélite blanco,
Que gira sobre la Tierra sin fin.
Con luz que brillando, llena el espacio,
Que nos llena de alegría y paz.
La luna, un tesoro que nos espera.

Texto refinado:
 La luna, un satélite blanco, girando sobre la Tierra sin fin, es un testimonio de la fascinación que nos rodea. Su luz radiante llena el espacio, creando un lienzo de alegría y paz. La luna, como un tesoro, espera nuestra atención, una regalo que hará que el alma se alegre de su presencia.
